In [1]:
from datasets import load_dataset

dataset = load_dataset("empathetic_dialogues", trust_remote_code=True)


README.md: 0.00B [00:00, ?B/s]

empathetic_dialogues.py: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/76673 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10943 [00:00<?, ? examples/s]

In [2]:
train_dataset = dataset["train"]
val_dataset = dataset["validation"]


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
from datasets import Dataset, DatasetDict
from tqdm import tqdm
from collections import defaultdict


def build_context_response_pairs(dataset):
    input_texts = []
    target_texts = []

    # Group utterances by conversation ID
    conversations = defaultdict(list)
    for example in dataset:
        conversations[example["conv_id"]].append(example)

    for conv in conversations.values():
        if len(conv) < 2:
            continue  # Skip conversations that are too short

        # Sort by utterance index to preserve order
        conv = sorted(conv, key=lambda x: x["utterance_idx"])

        # Determine speaker roles based on the first switch
        first_speaker = conv[0]["speaker_idx"]
        bot_speaker = None

        for utt in conv:
            if utt["speaker_idx"] != first_speaker:
                bot_speaker = utt["speaker_idx"]
                break

        if bot_speaker is None:
            continue  # Conversation never switches speakers

        user_speaker = first_speaker

        # Get the prompt only once
        prompt = conv[0]["prompt"]
        history = f"Prompt: {prompt}\nHistory: "
        Utterance = f"Utterance: "

        for i in range(len(conv)):
            speaker = conv[i]["speaker_idx"]
            utterance = conv[i]["utterance"]
            emotion = conv[i]["context"]

            # Assign role
            role = "Bot" if speaker == bot_speaker else "User"
            if role != "Bot":
                
                Utterance += f"{role}: {utterance}\n"

            # If it's a bot turn and there is at least one user input before
            if role == "Bot" and i > 0:
                context_text = history.strip()
                input_texts.append(f"{context_text}\n{Utterance.strip()}")

                target_texts.append(f"{emotion}\n{utterance}")
            if role == "Bot":
                Utterance = f"Utterance: "
            history += f"{role}: {utterance}\n"
            

    return Dataset.from_dict({"input": input_texts, "output": target_texts})

def tokenize(example):
    input_text = f"Context:\n{example['input']}\nResponse:\n"
    target_text = example["output"]

    # Combine input and target
    full_text = input_text + target_text

    # Tokenize the full sequence
    tokens = tokenizer(
        full_text,
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors=None
    )

    # Calculate how many tokens belong to input (to mask them from loss)
    input_ids_only = tokenizer(
        input_text,
        truncation=True,
        max_length=256,
        return_tensors=None
    )["input_ids"]

    input_len = len(input_ids_only)

    # Create label mask: -100 for input tokens, actual tokens for target
    labels = [-100] * input_len + tokens["input_ids"][input_len:]
    labels = labels[:256]  # Ensure same length as input_ids
    tokens["labels"] = labels

    return tokens


In [8]:
train_formatted = build_context_response_pairs(train_dataset)
val_formatted = build_context_response_pairs(val_dataset)

tokenized_train = train_formatted.map(tokenize)
tokenized_val = val_formatted.map(tokenize)

tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/36629 [00:00<?, ? examples/s]

Map:   0%|          | 0/5712 [00:00<?, ? examples/s]

In [9]:
for n in range(0, 8):
    print(train_formatted[n])

{'input': 'Prompt: I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.\nHistory: User: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.\nUtterance: User: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.', 'output': 'sentimental\nWas this a friend you were in love with_comma_ or just a best friend?'}
{'input': 'Prompt: I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.\nHistory: User: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ 

In [10]:
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained("distilgpt2")
model.resize_token_embeddings(len(tokenizer))  # Match tokenizer
model.to("cuda" if torch.cuda.is_available() else "cpu")


2025-06-26 05:54:23.013299: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750917263.274833      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750917263.353385      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [11]:
from transformers import TrainingArguments, EarlyStoppingCallback, Trainer

training_args = TrainingArguments(
    output_dir="./empathetic_bot_v2",
    num_train_epochs=4,                         # 👈 With large datasets, 1–2 epochs may be enough
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,              # 👈 Simulates larger effective batch size (32)
    
    eval_strategy="steps",
    eval_steps=1000,                            # 👈 Evaluate less frequently for large data
    save_strategy="steps",
    save_steps=1000,
    logging_steps=200,

    fp16=True,
    save_total_limit=2,
    load_best_model_at_end=True,

    weight_decay=0.01,                          # 👈 Regularization (L2)
    learning_rate=3e-5,                         # 👈 Conservative LR for stability
    lr_scheduler_type="cosine_with_restarts",   # 👈 Often generalizes better
    warmup_ratio=0.1,                           # 👈 Auto-scaling warmup

    report_to="none"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

In [12]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
1000,0.308600,0.333320
2000,0.293500,0.325321
3000,0.285300,0.322288
4000,0.276600,0.321297


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=4580, training_loss=0.4123884992307971, metrics={'train_runtime': 3510.3756, 'train_samples_per_second': 41.738, 'train_steps_per_second': 1.305, 'total_flos': 9571038677434368.0, 'train_loss': 0.4123884992307971, 'epoch': 4.0})

In [13]:
model.save_pretrained("empathetic-kaggle_v2")
tokenizer.save_pretrained("empathetic-kaggle_v2")


('empathetic-kaggle_v2/tokenizer_config.json',
 'empathetic-kaggle_v2/special_tokens_map.json',
 'empathetic-kaggle_v2/vocab.json',
 'empathetic-kaggle_v2/merges.txt',
 'empathetic-kaggle_v2/added_tokens.json',
 'empathetic-kaggle_v2/tokenizer.json')

In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the fine-tuned model from Kaggle path
model_path = "empathetic-kaggle_v2"  # Replace with correct path if different
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.to("cuda" if torch.cuda.is_available() else "cpu")

def chat():
    print("🤖 Empathetic Bot is ready. Type your message (type 'quit' to exit)\n")
    history = ""
    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            break
        if user_input.lower() == "new":
            history = ""
            continue

        # Format the chat history properly
        history += f"User: {user_input}\nBot:"

        inputs = tokenizer(history, return_tensors="pt", truncation=True, max_length=512).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=100,
                pad_token_id=tokenizer.eos_token_id,
                temperature=0.7,
                top_k=50,
                top_p=0.9,
                do_sample=True
            )

        full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract only the new response (after last "Bot:")
        response = full_output.split("Bot:")[-1].strip().split("User:")[0].strip()
        print(f"Bot: {response}\n")

        # Add bot reply to history
        history += f" {response}\n"

# Start the chatbot
chat()


🤖 Empathetic Bot is ready. Type your message (type 'quit' to exit)



You:  I am sad


Bot: That's sad. I am sorry to hear that.



KeyboardInterrupt: Interrupted by user

In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the fine-tuned model from Kaggle path
model_path = "empathetic-kaggle_v2"  # Replace with correct path if different
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.to("cuda" if torch.cuda.is_available() else "cpu")

def chat():
    print("🤖 Empathetic Bot is ready. Type your message (type 'quit' to exit)\n")
    history = ""
    MAX_HISTORY_LINES = 6
    
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ["quit", "exit"]:
            break
        if user_input.lower() in ["new", "reset"]:
            history = ""
            print("🔁 Context reset.\n")
            continue
    
        # Add latest user input
        history += f"User: {user_input}\nBot:"
    
        # Trim history
        history_lines = history.strip().split("\n")
        if len(history_lines) > MAX_HISTORY_LINES:
            history = "\n".join(history_lines[-MAX_HISTORY_LINES:])
    
        inputs = tokenizer(history, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    
        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_new_tokens=100,
                pad_token_id=tokenizer.eos_token_id,
                temperature=0.6,
                top_k=50,
                top_p=0.85,
                do_sample=False,
                repetition_penalty=1.1,
                no_repeat_ngram_size=3
            )
    
        full_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        response = full_output.split("Bot:")[-1].strip().split("User:")[0].strip()
    
        print(f"Bot: {response}\n")
        history += f" {response}\n"


# Start the chatbot
chat()


🤖 Empathetic Bot is ready. Type your message (type 'quit' to exit)



You:  I am sad


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Bot: That is terrible.



You:  My friend is angry with me.


Bot: Well_comma_ it's understandable that you feel the same way about others



You:  But I am not angry with him


Bot: It can be frustrating to have people like yourself who are just trying hard for themselves



KeyboardInterrupt: Interrupted by user

In [23]:
import shutil

# Compress the folder into a zip file
shutil.make_archive("empathetic-kaggle_v2", 'zip', "empathetic-kaggle_v2")

'/kaggle/working/empathetic-kaggle_v2.zip'